In [2]:
## Python web scrapping for Olympics races

In [3]:
!pip install pandas requests beautifulsoup4 lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 4.8 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/4.0 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 7.1 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# ============================================
# 1. BUILD ALL URLS PROGRAMMATICALLY
# ============================================

def build_event_pages():
    event_pages = []

    olympic_years = [2008, 2012, 2016, 2020]
    worlds_old_years = [2009, 2011, 2013, 2015, 2017]
    worlds_new_years = [2019, 2022, 2023]

    events = [
        ("100_metres", "100m"),
        ("200_metres", "200m"),
        ("400_metres", "400m"),
    ]

    genders = {
        "M": "Men%27s",
        "F": "Women%27s"
    }

    # Olympics
    for year in olympic_years:
        for dist_slug, dist_label in events:
            for gender_code, gender_slug in genders.items():
                url = (
                    f"https://en.wikipedia.org/wiki/"
                    f"Athletics_at_the_{year}_Summer_Olympics_%E2%80%93_"
                    f"{gender_slug}_{dist_slug}"
                )
                event_pages.append(
                    {
                        "event": dist_label,
                        "gender": gender_code,
                        "competition": "Olympics",
                        "year": year,
                        "url": url,
                    }
                )

    # World Championships – older naming
    for year in worlds_old_years:
        for dist_slug, dist_label in events:
            for gender_code, gender_slug in genders.items():
                url = (
                    f"https://en.wikipedia.org/wiki/"
                    f"{year}_World_Championships_in_Athletics_%E2%80%93_"
                    f"{gender_slug}_{dist_slug}"
                )
                event_pages.append(
                    {
                        "event": dist_label,
                        "gender": gender_code,
                        "competition": "World Championships",
                        "year": year,
                        "url": url,
                    }
                )

    # World Championships – newer naming
    for year in worlds_new_years:
        for dist_slug, dist_label in events:
            for gender_code, gender_slug in genders.items():
                url = (
                    f"https://en.wikipedia.org/wiki/"
                    f"{year}_World_Athletics_Championships_%E2%80%93_"
                    f"{gender_slug}_{dist_slug}"
                )
                event_pages.append(
                    {
                        "event": dist_label,
                        "gender": gender_code,
                        "competition": "World Championships",
                        "year": year,
                        "url": url,
                    }
                )

    return event_pages


EVENT_PAGES = build_event_pages()
print(f"Total Wikipedia pages to scrape: {len(EVENT_PAGES)}")
for p in EVENT_PAGES[:5]:
    print("  ", p["url"])

# ============================================
# 2. SCRAPER HELPERS
# ============================================

# global session with browser-like headers
SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
})

def clean_col(col):
    col = str(col).strip()
    col = col.replace("\xa0", " ")
    return col.lower()


def scrape_event_page(meta):
    """
    meta = dict with keys: event, gender, competition, year, url
    returns: list of DataFrames (one per relevant table) or empty list
    """
    url = meta["url"]
    print(f"\nScraping: {url}")
    try:
        r = SESSION.get(url, timeout=20)
        r.raise_for_status()
    except Exception as e:
        print(f"  !! Error requesting page: {e}")
        return []

    html = r.text

    try:
        tables = pd.read_html(html)
    except ValueError:
        print("  !! No tables found by pandas.read_html")
        return []

    dfs = []

    for idx, table in enumerate(tables):
        cols = [clean_col(c) for c in table.columns]

        has_lane = any("lane" in c for c in cols)
        has_perf = any(
            ("time" in c) or ("mark" in c) or ("result" in c)
            for c in cols
        )

        if not (has_lane and has_perf):
            continue

        df = table.copy()
        df.columns = cols

        df["event"] = meta["event"]
        df["gender"] = meta["gender"]
        df["competition"] = meta["competition"]
        df["year"] = meta["year"]
        df["round_raw"] = f"table_{idx}"

        dfs.append(df)

    if not dfs:
        print("  (no relevant lane/time tables on this page)")
    else:
        print(f"  -> found {len(dfs)} relevant table(s)")

    return dfs

# ============================================
# 3. RUN SCRAPER OVER ALL PAGES
# ============================================

all_tables = []

for meta in EVENT_PAGES:
    dfs = scrape_event_page(meta)
    all_tables.extend(dfs)

if not all_tables:
    print("\nNo lane/time tables found in any page. "
          "Check URLs or logic.")
    raise SystemExit

combined = pd.concat(all_tables, ignore_index=True)

# ============================================
# 4. NORMALIZE IMPORTANT COLUMNS
# ============================================

rename_map = {}

for col in combined.columns:
    low = col.lower()
    if "lane" in low and "lane" not in rename_map:
        rename_map[col] = "lane"
    elif ("pos" in low or "place" in low or "rank" in low) and "position" not in rename_map:
        rename_map[col] = "position"
    elif "athlete" in low or "name" in low:
        rename_map[col] = "athlete"
    elif "nation" in low or "country" in low or "noc" in low:
        rename_map[col] = "country"
    elif "reaction" in low:
        rename_map[col] = "reaction_time"
    elif "time" in low or "mark" in low or "result" in low:
        if "time_or_mark" not in rename_map.values():
            rename_map[col] = "time_or_mark"

combined = combined.rename(columns=rename_map)

keep_cols = [
    "event", "gender", "competition", "year",
    "round_raw", "lane", "position", "athlete",
    "country", "time_or_mark", "reaction_time"
]
keep_cols = [c for c in keep_cols if c in combined.columns]
combined = combined[keep_cols]

if "lane" in combined.columns:
    combined["lane"] = pd.to_numeric(combined["lane"], errors="coerce")
if "position" in combined.columns:
    combined["position"] = pd.to_numeric(combined["position"], errors="coerce")

combined = combined.dropna(subset=["lane", "time_or_mark"], how="any")

print("\n============================================")
print(f"Total rows collected after cleaning: {len(combined)}")
print("Sample rows:")
print(combined.head())
print("============================================\n")

output_file = "lane_results_advanced.csv"
combined.to_csv(output_file, index=False)
print(f"Saved cleaned dataset to: {output_file}")


Total Wikipedia pages to scrape: 72
   https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_100_metres
   https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Women%27s_100_metres
   https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_200_metres
   https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Women%27s_200_metres
   https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_400_metres

Scraping: https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_100_metres


C:\Windows\Temp\ipykernel_72008\3090015058.py:130: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


  -> found 18 relevant table(s)

Scraping: https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Women%27s_100_metres


C:\Windows\Temp\ipykernel_72008\3090015058.py:130: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


  -> found 3 relevant table(s)

Scraping: https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_200_metres


...

  -> found 1 relevant table(s)

Scraping: https://en.wikipedia.org/wiki/2023_World_Athletics_Championships_%E2%80%93_Men%27s_400_metres


C:\Windows\Temp\ipykernel_72008\3090015058.py:130: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


  (no relevant lane/time tables on this page)

Scraping: https://en.wikipedia.org/wiki/2023_World_Athletics_Championships_%E2%80%93_Women%27s_400_metres


C:\Windows\Temp\ipykernel_72008\3090015058.py:130: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


  (no relevant lane/time tables on this page)

Total rows collected after cleaning: 3416
Sample rows:
  event gender competition  year round_raw  lane  position          athlete  \
0  100m      M    Olympics  2008   table_7   3.0       1.0       Usain Bolt   
1  100m      M    Olympics  2008   table_7   9.0       2.0    Daniel Bailey   
2  100m      M    Olympics  2008   table_7   6.0       3.0  Vicente de Lima   
3  100m      M    Olympics  2008   table_7   2.0       4.0   Henry Vizcaíno   
4  100m      M    Olympics  2008   table_7   4.0       5.0    Fabio Cerutti   

  athlete athlete athlete              country country country time_or_mark  \
0     NaN     NaN     NaN              Jamaica     NaN     NaN        10.20   
1     NaN     NaN     NaN  Antigua and Barbuda     NaN     NaN        10.24   
2     NaN     NaN     NaN               Brazil     NaN     NaN        10.26   
3     NaN     NaN     NaN                 Cuba     NaN     NaN        10.28   
4     NaN     NaN     NaN   

In [6]:
output_file = "lane_results_advanced.csv"
combined.to_csv(output_file, index=False)


In [7]:
import os
os.getcwd()


'C:\\Users\\jai08'